In [1]:
import sys
sys.path.append('../compression-filter/src')
sys.path.append('../compression-filter')

import datetime
import os

import dotenv
import xarray

from earth2mip import inference_ensemble, registry
from earth2mip.initial_conditions import cds
from compressed_data_source import ZarrSource, CompressedWrapper
import numpy as np
from zarr_filter import J2KFilter
from filter_wrapper import JP2SPWV_Filter
import matplotlib.pyplot as plt

In [2]:
dlwp_vars = ['z1000', 'z925', 'z850', 'z700', 'z600', 'z500', 'z400', 'z300', 'z250', 'z200', 'z150', 'z100', 'z50', 'q1000', 'q925', 'q850', 'q700', 'q600', 'q500', 'q400', 'q300', 'q250', 'q200', 'q150', 'q100', 'q50', 't1000', 't925', 't850', 't700', 't600', 't500', 't400', 't300', 't250', 't200', 't150', 't100', 't50', 'u1000', 'u925', 'u850', 'u700', 'u600', 'u500', 'u400', 'u300', 'u250', 'u200', 'u150', 'u100', 'u50', 'v1000', 'v925', 'v850', 'v700', 'v600', 'v500', 'v400', 'v300', 'v250', 'v200', 'v150', 'v100', 'v50', 'msl', 'u10m', 'v10m', 't2m']
# dlwp_vars = ['q1000']#, 'q925', 'q850', 'q700', 'q600', 'q500', 'q400', 'q300', 'q250', 'q200', 'q150', 'q100', 'q50']
time = datetime.datetime(2017, 1, 2, 0)

In [3]:
filter_opts = JP2SPWV_Filter(
        base_cr=5000,
        height=721,
        width=1440).hdf_filter_opts

dlwp_data_source = cds.DataSource(dlwp_vars)
zarr_source = ZarrSource(dlwp_vars, '../../data/january.zarr')
compressed_source = CompressedWrapper(ZarrSource(dlwp_vars, '../../data/january.zarr'), J2KFilter(filter_opts, libpath='/scratch/lfusco/weather-benchmarks/compression-filter/src/build/lib/libh5z_j2k.so'))


In [4]:
filter_opts

(721, 1440, 1167867904, 0)

In [5]:
# data = dlwp_data_source[time]
zarr_data = zarr_source[time]
compressed_data = compressed_source[time]

In [6]:
zarr_data = np.array(zarr_data)

In [11]:
fig, axs = plt.subplots(3, len(dlwp_vars), figsize=(len(dlwp_vars) * 16, 20))
for i in range(len(dlwp_vars)):
    axs[0, i].set_title(dlwp_vars[i])
    cax = axs[0, i].imshow(zarr_data[i])
    fig.colorbar(cax, ax=axs[0, i])
    cax = axs[1, i].imshow(compressed_data[i])
    fig.colorbar(cax, ax=axs[1, i])
    cax = axs[2, i].imshow(compressed_data[i] - zarr_data[i])
    fig.colorbar(cax, ax=axs[2, i])

Error in callback <function _draw_all_if_interactive at 0x7fca8db8e480> (for post_execute), with arguments args (),kwargs {}:


ValueError: Image size of 110400x2000 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 110400x2000 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 110400x2000 with 414 Axes>

In [29]:
for i in range(7):
    print(np.max(np.abs(data[i] - compressed_data[i])), np.max(np.abs(data[i])))

11.01434326171875 302.5964660644531
45.9364013671875 4339.89453125
33.85546875 31925.87109375
32.64453125 58377.35546875
31.890625 95586.125
5.960866928100586 72.36321067810059
5.5894775390625 311.9093017578125


In [18]:
conditions = np.load('/scratch/lfusco/weather-benchmarks/outputs/10/initial_conditions.npy')
conditions = conditions.squeeze()

In [7]:
conditions = compressed_data

In [8]:
fig, axs = plt.subplots(3*3, 69//3, figsize=((69//3) * 8, 10 * 3))
for i in range(69):
    col = i % (69//3)
    row = i // (69//3)
    axs[row*3 + 0, col].set_title(dlwp_vars[i])
    cax = axs[row*3 + 0, col].imshow(zarr_data[i])
    fig.colorbar(cax, ax=axs[row*3 + 0, col])
    cax = axs[row*3 + 1, col].imshow(conditions[i])
    fig.colorbar(cax, ax=axs[row*3 + 1, col])
    cax = axs[row*3 + 2, col].imshow(conditions[i] - zarr_data[i])
    fig.colorbar(cax, ax=axs[row*3 + 2, col])